In [2]:
import spacy
from tika import parser
from spacy import displacy
import re
from spacy.matcher import PhraseMatcher
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.corpus import wordnet
#nltk.download('wordnet')

In [111]:
parsed = parser.from_file('Resume_1.pdf')
meta = parsed["metadata"]
st = re.sub(' +', ' ', parsed['content'].strip())
#st = re.sub(r"(\n){2,}", "\n", parsed['content'].strip())
#st = re.sub(r"( \n){2,}", "\n", st)           #\n{2,}|( \n){2,}", "\n", parsed['content'].strip())
st = re.sub(r'[^\x00-\x7F]+',' ', st)

nlp = spacy.load('en_core_web_sm')
doc = nlp(st)

In [112]:
def get_extension(dic):
    if 'language' in dic.keys():
        print(dic['language'])
    if 'pdf:docinfo:creator_tool' in dic.keys():
        print(dic['pdf:docinfo:creator_tool']) 
    if 'producer' in dic.keys():
        print(dic['producer'])
    if 'resourceName' in dic.keys():
        print(dic['resourceName'])
            
def extract_mail(text):
    emails = re.findall(r"[a-zA-Z0-9\.\-+_]+@[a-zA-Z0-9\.\-+_]+\.[A-Za-z]+", str(text))
    print(emails)
    
def extract_num(text):
    pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
                # Understanding the above regex
                # +91 or (91) -> [+(]? \d+ -?
                # Metacharacters have to be escaped with \ outside of character classes; inside only hyphen has to be escaped
                # hyphen has to be escaped inside the character class if you're not incidication a range
                # General number formats are 123 456 7890 or 12345 67890 or 1234567890 or 123-456-7890, hence 3 or more digits
                # Amendment to above - some also have (0000) 00 00 00 kind of format
                # \s* is any whitespace character - careful, use [ \t\r\f\v]* instead since newlines are trouble
    match = pattern.findall(str(text))
        # match = [re.sub(r'\s', '', el) for el in match]
        # Get rid of random whitespaces - helps with getting rid of 6 digits or fewer (e.g. pin codes) strings
        # substitute the characters we don't want just for the purpose of checking
    match = [re.sub(r'[,.]', '', el) for el in match if len(re.sub(r'[()\-.,\s+]', '', el))>6]
        # Taking care of years, eg. 2001-2004 etc.
    match = [re.sub(r'\D$', '', el).strip() for el in match]
        # $ matches end of string. This takes care of random trailing non-digit characters. \D is non-digit characters
    match = [el for el in match if len(re.sub(r'\D','',el)) <= 15]
        # Remove number strings that are greater than 15 digits        

    try:
        for el in list(match):
            # Create a copy of the list since you're iterating over it
            if len(el.split('-')) > 3: continue # Year format YYYY-MM-DD
            for x in el.split("-"):
                try:
                    # Error catching is necessary because of possibility of stray non-number characters
                    # if int(re.sub(r'\D', '', x.strip())) in range(1900, 2100):
                    if x.strip()[-4:].isdigit():
                        if int(x.strip()[-4:]) in range(1900, 2100):
                            # Don't combine the two if statements to avoid a type conversion error
                            match.remove(el)
                except:
                    pass
    except:
        pass
    return match

https://nerdgeeklab.com/nerdgeeklab/user_response.php?test=50

In [113]:
#Remove father name if present
matcher = PhraseMatcher(nlp.vocab)
phrase_list = ['father', 'father\'s', 'fathers', 'Father', 'FATHER', 'FATHER\'S', 'Father\'s', 'Fathers']
phrase_patterns = [nlp(text) for text in phrase_list]
matcher.add('fatherParser', None, *phrase_patterns)
matches = matcher(doc)

In [114]:
#Remove moter name if present
matcher2 = PhraseMatcher(nlp.vocab)
phrase_list2 = ['mother', 'mother\'s', 'mothers', 'Mother', 'MOTHER', 'MOTHER\'S', 'Mother\'s', 'Mothers']
phrase_patterns2 = [nlp(text) for text in phrase_list2]
matcher2.add('motherParser', None, *phrase_patterns2)
matches2 = matcher2(doc)

In [115]:
if matches2 == []:
    pass
else:
    mother_match = str(doc[matches2[0][1]:]).split('\n')[0]
    text = str(doc).replace(mother_match, ' ')
    doc = nlp(text)

if matches == []:
    pass
else:
    father_match = str(doc[matches[0][1]:]).split('\n')[0]
    text = str(doc).replace(father_match, ' ')
    doc = nlp(text)

In [116]:


Sentences = nltk.sent_tokenize(str(doc))
Tokens = []
for Sent in Sentences:
    Tokens.append(nltk.word_tokenize(Sent)) 
Words_List = [nltk.pos_tag(Token) for Token in Tokens]

Nouns_List = []

for List in Words_List:
    for Word in List:
        #[NN.*]
        
        if re.match('[NN.*]', Word[1]):
             Nouns_List.append(Word[0])

Names = []
for Nouns in Nouns_List:
    if not wordnet.synsets(Nouns):
        Names.append(Nouns)

name =[]
for n in Names:
    if re.search(r"\w", n):
        name.append(n)
name = list(map(str.lower, name))
print(name)

['abhishek', 'kumar', 'a129', 'vihar', 'mohan', 'pvt', '.cognizant', '.deloitte', '.untied', 'uhg', '.larsen', 'toubro', 'infotech', '.photography', 'etc', '.content', 'vedios', 'amoungs', 'socity', '.spid', '.spid', 'mmitr', '.spid', 'mahila', 'parmoter', 'dyal', 'b.com', 'sem', 'govt', 'govt', 'congnizant', 'fullstak', 'sfdc', 'sitecore', '-deloitte', 'sfdc', '-uhg', '-dsb', 'rights.and', 'smm', 'facebook', 'twiter', 'instagram', 'whatsapp']


In [117]:
with open('Childnames.txt', 'r') as myfile:
    data=myfile.read().replace('\n', ' ')

In [118]:
data = data.lower()
data = list(filter(None, data.split(" ")))

In [119]:
fi_names=[]
for n in name:
    if n in data:
        fi_names.append(n)    

In [120]:
fi_names

['abhishek', 'kumar', 'mohan', 'dyal']

In [121]:
doc = nlp(str(doc).lower())

In [122]:
from spacy.matcher import Matcher
matcher3 = Matcher(nlp.vocab)
pattern1 = [{'LOWER': fi_names[0]}]
matcher3.add('name_extract', None, pattern1)
found_matches = matcher3(doc)
print(found_matches)

[(17283935019124245801, 0, 1)]


In [123]:
final_name = str(doc[found_matches[0][1]:]).split('\n')[0]

In [124]:
final_name.strip()

'abhishek kumar'

In [108]:
get_extension(meta)

Resume_5.pdf


In [109]:
extract_mail(doc)

['jatinreaper@gmail.com']


In [110]:
extract_num(doc)

['9625485336']

In [ ]:
def text_summary():
    pass
    '''
    for more analysis
    '''

In [ ]:
def university():
    pass
    '''
    for univ details
    '''

In [ ]:
def skills():
    pass
    '''
    skills
    '''

# Anything else??? Will work on it later